In [1]:
import shutil
from icecream import ic
import glob
import pandas as pd

In [2]:
ORIGINAL_DATASET_PATH='/media/aubrey/Extreme SSD/dataset2'
NEW_DATASET_PATH='/home/aubrey/dataset3'

# Delete NEW_DATASET_PATH to start from scratch

ic('copying dataset')
try:
    shutil.copytree(ORIGINAL_DATASET_PATH, NEW_DATASET_PATH)
except:
    pass


ic| 'copying dataset'


In [3]:
!du -h {NEW_DATASET_PATH}

186M	/home/aubrey/dataset3/labels/train
24M	/home/aubrey/dataset3/labels/val
23M	/home/aubrey/dataset3/labels/test
232M	/home/aubrey/dataset3/labels
6.4G	/home/aubrey/dataset3/images/train
803M	/home/aubrey/dataset3/images/val
806M	/home/aubrey/dataset3/images/test
7.9G	/home/aubrey/dataset3/images
8.2G	/home/aubrey/dataset3


In [4]:

ic('getting list of annotation files (*.txt)')
search_string = f'{NEW_DATASET_PATH}/labels/**/*.txt'
annotation_file_list = glob.glob(search_string, recursive=True)
ic(len(annotation_file_list))

ic('removing bounding boxes')
total_bad_bbs = 0
for i, annotation_file in enumerate(annotation_file_list):
    # read file
    df = pd.read_csv(annotation_file, header=None, sep=' ', names=['cls','xc','yc','width','height'])
    
    # calculate left, right and top coordinates for bounding boxes
    df['left'] = df.xc - (df.width / 2)
    df['right'] = df.xc + (df.width / 2)
    df['top'] = df.yc - (df.height / 2)
    
    # create a new df containing bounding boxes which do net touch left, right or top of image
    df1 = df[(df.left > 0.001) & (df.right < 0.999) & (df.top > 0.001)]

    # if bounding boxes touching left, right or top edges of image were detected
    # update the annotation file
    bad_bbs = df.shape[0] - df1.shape[0]
    if bad_bbs > 0:
        total_bad_bbs += bad_bbs
        df1.to_csv(annotation_file, 
                   sep=' ', index=False, header=False, columns=['cls','xc','yc','width','height']) 
    if i % 10000 == 0:
        ic(i, total_bad_bbs)
       
ic(i, total_bad_bbs)

ic| 'getting list of annotation files (*.txt)'
ic| len(annotation_file_list): 70888
ic| 'removing bounding boxes'
ic| i: 0, total_bad_bbs: 0
ic| i: 10000, total_bad_bbs: 0
ic| i: 20000, total_bad_bbs: 0
ic| i: 30000, total_bad_bbs: 0
ic| i: 40000, total_bad_bbs: 0
ic| i: 50000, total_bad_bbs: 0
ic| i: 60000, total_bad_bbs: 0
ic| i: 70000, total_bad_bbs: 0
ic| i: 70887, total_bad_bbs: 0


(70887, 0)

In [5]:
!du -h {NEW_DATASET_PATH}

186M	/home/aubrey/dataset3/labels/train
24M	/home/aubrey/dataset3/labels/val
23M	/home/aubrey/dataset3/labels/test
232M	/home/aubrey/dataset3/labels
6.4G	/home/aubrey/dataset3/images/train
803M	/home/aubrey/dataset3/images/val
806M	/home/aubrey/dataset3/images/test
7.9G	/home/aubrey/dataset3/images
8.2G	/home/aubrey/dataset3
